# Cambiamenti climatici

## 1. Data collection
I dati usati in questa applicazione si trovano nella cartella [data](applicazione/data/). 
<ul>
    <li>In original si trovano i file originali trovati su internet</li>
    <li>In modified si trovano i file "processati"</li>
</ul>

## Dati sulle temperature e sulle stazioni geografiche
I file contenenti i dati relativi alle temperature ([temperatures.dat](applicazione/data/original/temperatures.dat)) e alle stazioni geografiche ([stations.inv](applicazione/data/original/stations.inv)) sono stati trovati sul sito [National Centers for Environmental Information](https://www.ncei.noaa.gov/). Sono scaricabili al seguente link: https://www.ncei.noaa.gov/pub/data/ghcn/v4/ghcnm.tavg.latest.qcu.tar.gz <br>


## Dati sui Paesi
I file contenenti informazioni sui Paesi sono stati trovati su due repository di github: <br>
- [fips_to_iso_country_codes.csv](applicazione/data/original/fips_to_iso_country_codes.csv) trovato al seguente link: https://github.com/mysociety/gaze/blob/master/data/fips-10-4-to-iso-country-codes.csv <br>
- [countries_info.csv](applicazione/data/original/countries_info.csv) trovato al seguente link https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv <br>

## 2. Data processing

In [186]:
import pandas as pd
import os

DATA_FOLDER = "applicazione" + os.path.sep + "data" + os.path.sep
DATA_ORIGINAL_FOLDER = DATA_FOLDER + "original"+ os.path.sep
DATA_MODIFIED_FOLDER = DATA_FOLDER + "modified"+ os.path.sep

def aggiungi_linea_iniziale(path_file1, path_file2, headers="-"):
    """Funzione che inserisce nel file path_file2 una linea vuota e il contenuto del file path_file1 """
    import time
    start_time = time.time()
    if path_file1 == path_file2:
        with open(path_file1, 'r') as file: 
            originalContent = file.read() 
        with open(path_file2, 'w') as file2:
            file2.write(headers + "\n" + originalContent)
    else:
        with open(path_file1, 'r') as file, open(path_file2, 'w') as file2: 
            originalContent = file.read() 
            file.seek(0, 0) # Prima posizione            
            file2.write(headers + "\n")
            file2.write(originalContent) 
    # print("--- %s seconds ---" % (time.time() - start_time))

def crea_file_csv(file_originale, file_modificato, list_width, headers="", column_to_drop=-1, crea_campo_fips=False):
    """Funzione che crea un file CSV partendo da un Fixed Width Text File. 
    In input sono richiesti: 
    - Il path del file originale
    - Il path del file CSV che verra' creat
    - Lista delle lunghezze delle colonne di file_originale"""
    if os.path.exists(file_modificato):
        return
    print("Creo file csv partendo da",file_originale,"...")
    file_tmp = file_originale + "tmp"
    aggiungi_linea_iniziale(file_originale, file_tmp)
    df = pd.read_fwf(file_tmp, widths=list_width)
    if column_to_drop != -1:
        df = df.drop(df.columns[column_to_drop], axis=1) 
    if crea_campo_fips:
        df['fips'] = df[df.columns[0]].apply(lambda x : x[0:2])
    df.to_csv(file_modificato, index=False, header=False)
    if headers != "":
        aggiungi_linea_iniziale(file_modificato, file_modificato, headers)

    os.remove(file_tmp)

### Stazioni e temperature
I file [stations.inv](applicazione/data/original/stations.inv) e [temperatures.dat](applicazione/data/original/temperatures.dat) trovati su internet sono Fixed Width Text File. Ho deciso di modificarli e convertirli in file CSV ([stations_created.csv](applicazione/data/modified/stations_created.csv), [tempetatures_created.csv](applicazione/data/modified/tempetatures_created.csv)) con la seguente struttura: <br>

#### stations_created.csv

|Campo| Significato|
|:-|:-|
|id_station| Codice identificativo della stazione. I primi due caratteri sono il codice FIPS del Paese|
|latitude| Latitudine della stazione in gradi decimali|
|longitude| Longitudine della stazione in gradi decimali|
|st_elev| Altitudine della stazione in metri; il valore -999.0 indica un dato mancante|
|name| Nome della stazione|
|fips| Codice FIPS del Paese|

#### tempetatures_created.csv

|Campo| Significato|
|:-|:-|
|id_station| Codice identificativo della stazione. I primi due caratteri sono il codice FIPS del Paese|
|year| Anno in cui sono state misurate le temperature|
|month_value| Valore della temperatura del mese "month". I valori della temperatura sono in centesimi di grado Celsius, ma sono espressi come numeri interi (ad es. dividere per 100,0 per ottenere gradi Celsius interi). <br>Il valore -9999 indica un dato mancante. |
|month_dm_flag| data measurement flag (indicatore della misurazione dei dati). <br>Se è vuoto -> nessuna informazione di misurazione applicabile <br> Se è una lettera tra a-i -> numero di giorni mancanti nel calcolo della temperatura media mensile |
|month_qc_flag| quality control flag del mese "month". <br>Se è vuoto, non c'è stato nessun fallimento del controllo di qualità oppure il valore non può essere valutato. <br> Per gli altri valori, consultare la [documentazione dei dataset](https://www.ncei.noaa.gov/pub/data/ghcn/v4/readme.txt) |
|month_ds_flag| Indica l'origine dati per il valore del mese "month". Per avere altre informazioni, consultare la [documentazione di tale campo](https://www.ncei.noaa.gov/pub/data/ghcn/v4/ghcnm-flags.txt)|

Ho creato il seguente script per creare i due file CSV:

In [188]:
import calendar

# File con le stazioni
FILE_ORIGINALE_STATIONS = DATA_ORIGINAL_FOLDER + "stations.inv"
FILE_CSV_STATIONS = DATA_MODIFIED_FOLDER + "stations_created.csv"
headers ="id_station,latitude,longitude,st_elev,name,fips"
crea_file_csv(FILE_ORIGINALE_STATIONS, FILE_CSV_STATIONS, [11,9,10,7,31], headers, crea_campo_fips=True)

# File con le temperature
FILE_ORIGINALE_TEMPERATURES = DATA_ORIGINAL_FOLDER + "temperatures.dat"
FILE_CSV_TEMPERATURES = DATA_MODIFIED_FOLDER + "temperatures_created.csv"

list_width= [11,4,4]
for _ in range(12):
    list_width += [5,1,1,1]

list_headers = ["id_station", "year","elem"]
for i in range(1,13):
    month_name = calendar.month_name[i]
    list_headers += ["month:"+month_name+"_value","month:"+month_name+"_dm_flag","month:"+month_name+"_qc_flag","month:"+month_name+"_ds_flag"]
headers = ",".join(list_headers)
crea_file_csv(FILE_ORIGINALE_TEMPERATURES, FILE_CSV_TEMPERATURES, list_width, headers,3)

print("Files creati correttamente!")

Creo file csv partendo da applicazione\data\original\stations.inv ...
Creo file csv partendo da applicazione\data\original\temperatures.dat ...
Files creati correttamente!


### Paesi
Per quanto riguarda le informazioni sui paesi, ho unito i due file [country_info.csv](applicazione/data/original/countries_info.csv) e [fips_to_iso_country_codes.csv](applicazione/data/original/fips_to_iso_country_codes.csv) in un unico file [countries_created.csv](applicazione/data/original/countries_created.csv) con la seguente struttura:

|Campo|Significato|
|:-|:-|
|fips| Codice FIPS del paese|
|name_country| Nome del Paese|
|region| Continente di appartenenza|
|sub-region| Zona del continente (nord, sud, est, ovest)|
|region-code| Codice di region|
|sub-region-code| Codice di sub-region|

In [189]:
pd.set_option('display.max_colwidth', None)

FILE_ORIGINALE_FIPS_TO_ISO = DATA_ORIGINAL_FOLDER + "fips_to_iso_country_codes.csv"
FILE_ORIGINALE_COUNTRY_INFO = DATA_ORIGINAL_FOLDER + "countries_info.csv"
FILE_CSV_COUNTRIES_CREATED = DATA_MODIFIED_FOLDER + "countries_created.csv"

# Leggo i files originali
df_fips = pd.read_csv(FILE_ORIGINALE_FIPS_TO_ISO)
df_country = pd.read_csv(FILE_ORIGINALE_COUNTRY_INFO)

# Li unisco in base al codice alpha
merged = pd.merge(df_fips, df_country, how='left',left_on='ISO 3166', right_on='alpha-2')

# Elimino le colonne non necessarie e rinomino le altre
merged = merged.drop(['ISO 3166','iso_3166-2', 'alpha-3', 'alpha-2', 'country-code', 'intermediate-region', 'intermediate-region-code'], axis=1)
merged['name_country'] = merged.apply(lambda x : str(x['Name'] if str(x["name"]).lower() == "nan" else x['name']), axis=1)
merged = merged.drop([ 'name', 'Name'], axis=1)
merged = merged.rename(columns={"FIPS 10-4":"fips"})

# e creo un nuovo file
merged.to_csv(FILE_CSV_COUNTRIES_CREATED, index=False, header=True)

print("File creato correttamente!")

File creato correttamente!


In [ ]:

# for index, row in merged.iterrows():
#     print(f"{index:5}", f"{row['name']:50}",row['fips'])


# print(merged)

df_stations = pd.read_csv(FILE_CSV_STATIONS)
# print(len(pd.unique(df_stations['fips'])))

merged2 = df_stations.merge(merged, how='left', on='fips')
# merged2 = merged2.drop(['name_country', 'region', 'sub-region', 'region-code', 'sub-region-code'], axis=1)
merged2 = merged2.drop_duplicates() 
merged2 = merged2.reset_index()

# print(merged2)
for index, row in merged2.iterrows():
    print(str(row).replace("\n","\t",-1))

